In [37]:
import pandas as pd 
import requests
from zipfile import ZipFile, BadZipFile
from io import BytesIO
from datetime import datetime

In [38]:
# # User inputs
# queryname = input('What are you looking for? ') # how to make a dropdown later?
# if queryname == 'option 1':
#     queryname = 'code' # repeat as many times as needed to fix codes after input

# startdate = input('Enter a startdate (MM/DD/YYYY): ')
# try:
#     date_obj = datetime.strptime(startdate, '%m/%d/%Y')
#     print('correct date format. Do you want to start at a specific time? ') # add in time settings later
# except ValueError:
#     print('invalid date format. Plase use MM/DD/YYYY') # create a loop probs

# enddate = input('Enter a startdate (MM/DD/YYYY): ')
# try:
#     date_obj = datetime.strptime(enddate, '%m/%d/%Y')
#     print('correct date format. Do you want to start at a specific time? ') # add in time settings later
# except ValueError:
#     print('invalid date format. Plase use MM/DD/YYYY')

# market_run_id = input('Day Ahead Market or Real Time? (DAM or RTM): ') # create a loop probs
# if market_run_id == 'RTM':
#     market_run_id = 'actual rtm code' # fix this later

# node = input('OPTIONAL: Specify a node. Separate multiple nodes with a comma. Leave blank for all nodes: ')
# if node == '': # means they want all nodes
#     node = 'ALL_APNODES' # what you put in to query all nodes
# if node != '':
#     node = node.replace(' ', '') # removing any spaces
#     # they have specified a node, but then I have to run the parameters differently. maybe I can put the parameters under this selection thing. Node will be last selection

Currently not going to run the cell above because it isn't done, and it needs a lot more code. It is a holding spot for eventual user interaction.

In [39]:
# setting parameters
base_url = "http://oasis.caiso.com/oasisapi/SingleZip"

# Define your query parameters as a dictionary
params = {
    "resultformat": "6", # should always be this- creates a CSV
    "queryname": "PRC_LMP", # determines what kind of data pull we want
    "startdatetime": "20250501T07:00-0000",
    "enddatetime": "20250502T07:00-0000",
    "market_run_id": "DAM",
    "version": "1", # version 1 is the oldest and the safest I think
    # "grp_type": "", # Optional parameter (can vary by query) does it switch to node if I include one?
    "node": "0096WD_7_N001,10TH_STW_1_1LNODE"
}

can't have node and grouptype in the same parameter- it breaks it. one or the other is okay

Query from the website: 
https://oasis.caiso.com/oasisapi/SingleZip?
resultformat=6&
queryname=PRC_LMP&
version=12
&startdatetime=20250501T07:00-0000&
enddatetime=20250502T07:00-0000&
market_run_id=DAM&
node=0096WD_7_N001,10TH_STW_1_1LNODE # two nodes!! separate with a comma

In [40]:
# calling AP
try: # using this for now for error handling
    response = requests.get(base_url, params=params)
    with ZipFile(BytesIO(response.content)) as z:
        for filename in z.namelist():
            with z.open(filename) as f:
                df = pd.read_csv(f)
                df.to_csv('test2.csv')
                # print(df.info())
                print(df.columns)
                # print(df['LMP_TYPE'].value_counts())
                # print(df['XML_DATA_ITEM'].value_counts)
except BadZipFile:
    print('Response is not a ZIP')
    print(response.content.decode(errors='replace'))


Index(['INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 'OPR_DT', 'OPR_HR',
       'OPR_INTERVAL', 'NODE_ID_XML', 'NODE_ID', 'NODE', 'MARKET_RUN_ID',
       'LMP_TYPE', 'XML_DATA_ITEM', 'PNODE_RESMRID', 'GRP_TYPE', 'POS', 'MW',
       'GROUP'],
      dtype='object')
